# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Set the path to citis.csv
cities_path = "../output_data/cities.csv"

# Read the cities.csv
cities_df = pd.read_csv(cities_path)

# # Set the index to the City ID
cities_df = cities_df.set_index('City ID')

# output cities_df
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
0,Bluff,93,NZ,1603217426,92,-46.60,168.33,41.00,1.99
1,São João da Barra,75,BR,1603217426,65,-21.64,-41.05,78.80,16.11
2,Ushuaia,40,AR,1603217130,60,-54.80,-68.30,42.80,20.80
3,Ponta do Sol,18,CV,1603217426,52,17.20,-25.09,89.60,20.80
4,High Level,40,CA,1603217366,100,58.52,-117.14,12.20,2.24
...,...,...,...,...,...,...,...,...,...
520,Ryōtsu-minato,0,JP,1603218098,59,38.08,138.43,60.04,9.46
521,Talakan,100,RU,1603218099,78,50.32,130.37,39.25,8.72
522,Bristol,90,US,1603218084,82,41.83,-71.17,66.20,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Define weights
weights = cities_df['Humidity'].astype(float)

# Genearte figure
cities_fig = gmaps.figure()

# Define heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# Add heat layer to figure
cities_fig.add_layer(heat_layer)

# Show the heatmap
cities_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Drop any null values
cities_df = cities_df.dropna()

# Setup variables for searching for the perfect vacation spot
low_temp = 70
high_temp = 80
max_wind_speed = 10
cloudiness = 0
humidity = 35

# Create a DataFrame with only cities that have our ideal weather
vacation_cities_df = cities_df.loc[cities_df['Wind Speed'] < max_wind_speed]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Cloudiness'] ==  cloudiness]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] > low_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] < high_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Humidity'] < humidity]
vacation_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
155,Hīt,0,IQ,1603217626,25,33.64,42.83,78.82,8.86
215,Naraura,0,IN,1603217692,21,28.20,78.38,73.98,3.09
264,Nizwá,0,OM,1603217763,21,22.93,57.53,72.70,2.66
378,Kulachi,0,PK,1603217962,21,31.93,70.46,70.81,1.39
458,Sakaraha,0,MG,1603218029,33,-22.90,44.53,76.26,4.18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create a new dataframe and add a Hotel Name column
hotel_df = pd.DataFrame(vacation_cities_df,
                      columns=['City','Country','Lat','Lng','Hotel Name'])

hotel_df

,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
155,Hīt,IQ,33.64,42.83,NaN
215,Naraura,IN,28.20,78.38,NaN
264,Nizwá,OM,22.93,57.53,NaN
378,Kulachi,PK,31.93,70.46,NaN
458,Sakaraha,MG,-22.90,44.53,NaN


In [6]:
# find the closest hotel for all of our possible vacation cities

# Define the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through possible vacation cities
for index, row in hotel_df.iterrows():
    
    # Set a variable for the latitude and Longitude of our city
    city_lat_lng = f"{row['Lat']},{row['Lng']}"
    
    # Define the api call parameters
    params = {
               "location": city_lat_lng,
               "radius": 5000,
               "rankby": "distance",
               "types": "lodging",
               "key": g_key
             }
    # Output that we are retrieving information for our city
    print(f"Retrieving Results for Index {index}: {row['City']}, {row['Country']}.")
   
    # assemble url and make API request
    response_json = requests.get(base_url, params=params).json()
    
    # extract results
    results = response_json['results']

    # If we got a hotel add it to the DataFrame
    # if we don't get a hotel then output a message that we are skipping it
    try:
        # Output the closet hotel
        print(f"Closest hotel is {results[0]['name']}.")
        
        # Update the data frame with the hotel name
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
 
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-----------------------------------------------------")

# Output hotel_df
hotel_df   

Retrieving Results for Index 155: Hīt, IQ.
Missing field/result... skipping.
-----------------------------------------------------
Retrieving Results for Index 215: Naraura, IN.
Missing field/result... skipping.
-----------------------------------------------------
Retrieving Results for Index 264: Nizwá, OM.
Missing field/result... skipping.
-----------------------------------------------------
Retrieving Results for Index 378: Kulachi, PK.
Missing field/result... skipping.
-----------------------------------------------------
Retrieving Results for Index 458: Sakaraha, MG.
Missing field/result... skipping.
-----------------------------------------------------


,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
155,Hīt,IQ,33.64,42.83,NaN
215,Naraura,IN,28.20,78.38,NaN
264,Nizwá,OM,22.93,57.53,NaN
378,Kulachi,PK,31.93,70.46,NaN
458,Sakaraha,MG,-22.90,44.53,NaN


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
cities_fig.add_layer(markers)

# Display Map
cities_fig

Figure(layout=FigureLayout(height='420px'))